# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd

In [2]:
# load data from database
from sqlalchemy import create_engine
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql('SELECT * FROM InsertTableName', con=engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
#Define feature and target variables X and Y
import numpy as np
X = df['message']
y = df.iloc[:, 4:]
y['related'].replace(2, 1, inplace=True)

### 2. Write a tokenization function to process your text data

In [4]:
import nltk
import re
nltk.download(['punkt', 'stopwords','wordnet'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
def tokenize(text):
    text = re.sub(r'[^\w\s]','',text.lower())
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tok = [lemmatizer.lemmatize(tok) for tok in tokens if tok not in stopwords.words("english")]

    return clean_tok

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
for message in X[:5]:
    print(message)
    print(tokenize(message))

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']
Is the Hurricane over or is it not over
['hurricane']
Looking for someone but no name
['looking', 'someone', 'name']
UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '8090', 'destroyed', 'hospital', 'st', 'croix', 'functioning', 'need', 'supply', 'desperately']
says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)), 
                    ('tfidf', TfidfTransformer()),
                    ('clf', MultiOutputClassifier(RandomForestClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42)

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

y_pred = pipeline.predict(X_test)

y_pred_df = pd.DataFrame(y_pred, columns = y_test.columns)

evaluation = {}

for column in y_test.columns:
    evaluation[column] = []
    evaluation[column].append(precision_score(y_test[column], y_pred_df[column]))
    evaluation[column].append(recall_score(y_test[column], y_pred_df[column]))
    evaluation[column].append(f1_score(y_test[column], y_pred_df[column]))

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [9]:
pd.DataFrame(evaluation)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0.845018,0.776000,0.0,0.740614,0.620690,0.730769,0.666667,0.0,0.538462,0.0,...,0.0,0.666667,0.801619,0.882353,0.719424,1.000000,0.844444,0.833333,0.700000,0.720339
1,0.906931,0.425439,0.0,0.597796,0.086124,0.137681,0.063492,0.0,0.087500,0.0,...,0.0,0.017857,0.572254,0.280374,0.425532,0.043478,0.730769,0.094340,0.053435,0.320151
2,0.874881,0.549575,0.0,0.661585,0.151261,0.231707,0.115942,0.0,0.150538,0.0,...,0.0,0.034783,0.667791,0.425532,0.534759,0.083333,0.783505,0.169492,0.099291,0.443286


In [10]:
# Classification report
for i in range(35):
    print("Column: {}".format(y_test.columns[i]))
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Column: related
             precision    recall  f1-score   support

          0       0.59      0.44      0.50       602
          1       0.85      0.91      0.87      2020

avg / total       0.79      0.80      0.79      2622

Column: request
             precision    recall  f1-score   support

          0       0.89      0.97      0.93      2166
          1       0.78      0.43      0.55       456

avg / total       0.87      0.88      0.86      2622

Column: offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      2608
          1       0.00      0.00      0.00        14

avg / total       0.99      0.99      0.99      2622

Column: aid_related
             precision    recall  f1-score   support

          0       0.75      0.85      0.80      1533
          1       0.74      0.60      0.66      1089

avg / total       0.75      0.75      0.74      2622

Column: medical_help
             precision    recall  f1-score   support


### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__estimator__

In [12]:
from sklearn.model_selection import GridSearchCV

parameters = {'vect__max_df': [0.75, 1.0]}

cv = GridSearchCV(pipeline, param_grid = parameters, verbose = 3, cv = 2, n_jobs = -1)

model = cv.fit(X_train, y_train)
y_pred = model.predict(X_test)

Fitting 2 folds for each of 2 candidates, totalling 4 fits
[CV] vect__max_df=0.75 ...............................................
[CV] ...... vect__max_df=0.75, score=0.2375180130541663, total= 2.3min
[CV] vect__max_df=0.75 ...............................................


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  3.2min remaining:    0.0s


[CV] ..... vect__max_df=0.75, score=0.23700940917182334, total= 2.3min
[CV] vect__max_df=1.0 ................................................


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  6.4min remaining:    0.0s


[CV] ...... vect__max_df=1.0, score=0.23616173603458507, total= 2.3min
[CV] vect__max_df=1.0 ................................................
[CV] ...... vect__max_df=1.0, score=0.24150207679918623, total= 2.2min


[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed: 12.6min finished


In [13]:
print('Best Parameters: {}'.format(model.best_params_))

Best Parameters: {'vect__max_df': 1.0}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
print('Accuracy:')
print((y_pred == y_test).mean())
for i in range(35):
    print("Column: {}".format(y_test.columns[i]))
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

Accuracy:
related                   0.807780
request                   0.877193
offer                     0.994661
aid_related               0.747140
medical_help              0.925629
medical_products          0.952326
search_and_rescue         0.975591
security                  0.982838
military                  0.969870
child_alone               1.000000
water                     0.955759
food                      0.931732
shelter                   0.924867
clothing                  0.985507
money                     0.983982
missing_people            0.988177
refugees                  0.967582
death                     0.959573
other_aid                 0.866514
infrastructure_related    0.933257
transport                 0.954233
buildings                 0.950420
electricity               0.978261
tools                     0.994661
hospitals                 0.988177
shops                     0.995805
aid_centers               0.985126
other_infrastructure      0.956522
weather_re

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [15]:
from sklearn.ensemble import AdaBoostClassifier

pipeline = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', MultiOutputClassifier(AdaBoostClassifier()))])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

print('Accuracy:')
print((y_pred == y_test).mean())
for i in range (35):
    print("Column: {}".format(y_test.columns[i]))
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

Accuracy:
related                   0.800153
request                   0.887872
offer                     0.993898
aid_related               0.749428
medical_help              0.925629
medical_products          0.953852
search_and_rescue         0.976354
security                  0.981312
military                  0.971014
child_alone               1.000000
water                     0.962243
food                      0.943173
shelter                   0.941648
clothing                  0.988558
money                     0.977879
missing_people            0.989321
refugees                  0.967582
death                     0.966438
other_aid                 0.866514
infrastructure_related    0.931350
transport                 0.955759
buildings                 0.957285
electricity               0.980931
tools                     0.993516
hospitals                 0.988177
shops                     0.994661
aid_centers               0.981693
other_infrastructure      0.952708
weather_re

### 9. Export your model as a pickle file

In [16]:
import pickle
filename = 'finalized_model.pkl'
pickle.dump(pipeline, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.